### Market Level Forecasting

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import preprocessing
from preprocessing import preprocessingMLS, featureEngineering
from getGoogleTrends import getGoogleTrendsData
import models
from imp import reload
import matplotlib.ticker as ticker

### Step 1: Read and preprocess data
House market data, Google Trends, and Economic indicators

In [3]:
data_dir = 'index-team-data/denver/'

In [4]:
#read in market data
process1 = preprocessingMLS(data_dir+'listing_dates_with_ct.csv')
monthData = process1.get_monthly_data(columns=['count_sale','count_list','sale_price'],start_date='2016-03', end_date='2020-09')


#get google trends data from API/read in data
#----------------------------------------------------
#gt = getGoogleTrendsData(geo='US-GA',city='Atlanta')
#trend = gt.getData(times=10,date='2016-03-01 2020-09-30',scaled=True)
#trend.to_csv('google_trends_atlanta_scaled.csv')
#----------------------------------------------------
trend = pd.read_csv(data_dir+'google_trends/google_trends_scaled.csv')


#read in economics indicator features, if needed
#econ = pd.read_csv('econ_data.csv')

merged = pd.merge(monthData, trend).drop(['Unnamed: 0'],axis=1)

In [5]:
#create lag features, pct change features and one hot month features with featureEngineering
feature_column_names = merged.columns.delete(0)
fe = preprocessing.featureEngineering(merged,feature_column_names)
fe.create_lag_features(lag_num_list=[1,3])
fe.create_pct_change_features(lag_num_list=[1])
fe.create_month_one_hot()
df = fe.get_dataFrame()
feature_column_names = fe.get_feature_names()
print(len(feature_column_names))

136


### Step 2: Make predictions

### 2.1 predict 3 months ahead -- google trends

In [6]:
# predict 3 months ahead -- google trends
target_col = 'count_sale'
horizon = 3
model = models.rollingModel(df,horizon,target_col,feature_column_names,
                            modelName='linear',lead_target=True)
pred, pred_boot, pred_scale, pred_samples = model.fit_predict(df[feature_column_names], df[target_col])

prediction_horizon 1


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4722.644282732159, tolerance: 3175.6425902439023
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5201.238530062139, tolerance: 3175.6425902439023
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3545.609328328108, tolerance: 3175.6425902439023
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to incre

prediction_horizon 2


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19978.727588786747, tolerance: 9044.668472000001
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21974.356792810617, tolerance: 7372.422618000001
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12408.438549723329, tolerance: 9140.119568
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase th

prediction_horizon 3


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 59805.237529705715, tolerance: 21140.844069387756
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 45289.57549991352, tolerance: 15525.839000000004
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 44552.518033237095, tolerance: 12751.6824122449
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to incre

### 2.2 predict 3 months ahead -- time series

In [7]:
#predict 3 months ahead -- Time series
model = models.BTSM()
model.fit(df['count_sale'].values.astype(float),0)
pred_ts, pred_ts_scale, pred_ts_samples = model.predict(np.ones(3))

Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.
Instructions for updating:
Do not call `graph_parents`.
Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Previously, `initial_step` would return the value passed to the constructor, even if that value was not `Tensor`-like. This behavior is now deprecated. In the future, it will always return a `Tensor`-like object.
Instructions for updating:
The signature for `trace_fn`s passed to `minimize` has changed. Trace functions now take a single `traceable_quantities` argument, which is a `tfp.math.MinimizeTraceableQuantities` namedtuple containing `traceable_quantities.loss`, `traceable_quantities.gradients`, etc. Please update your `trace_

### 2.3 predict 3 months ahead -- linear combination

In [8]:
#combine the two prediciton
#params may differ in different market
params = [0.6,0.4]
pred_com = params[0]*np.array(pred)+params[1]*pred_ts
pred_com_scale = params[0]*pred_scale+params[1]*pred_ts_scale

print(pred_com)

[7878.82040817 6472.99039788 6418.25346059]


### 2.4 predict 6 months ahead  -- number of sales

In [9]:
#predict 6 months ahead
model = models.rollingCombinedModel(df=df,predict_horizon_total=6, predict_horizon=3,
                                    feature_column_names = feature_column_names,
                                   target_column = target_col, modelName='linear',
                                   lead_target=True)

In [10]:
pred6_sale, pred6_scale_sale, pred6_samples_sale = model.make_prediction(df[feature_column_names], df[target_col])

prediction_horizon 1


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6819.6610656392295, tolerance: 5230.124219512195
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4152.680347692454, tolerance: 3445.941848780488
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18338.320745861136, tolerance: 4156.255592156863
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increa

prediction_horizon 2


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14869.39881719357, tolerance: 7851.324378
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 32866.061749653345, tolerance: 8731.826442000001
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18707.496556528233, tolerance: 8185.630377999999
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the

prediction_horizon 3


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 40994.80172495371, tolerance: 13945.118040816329
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 80867.0465400726, tolerance: 18587.46398367347
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 31540.61771729226, tolerance: 16520.640812244903
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increas

In [11]:
pred6_samples_sale.shape

(50, 6)

### 2.5 predict 6 months ahead  -- number of new listings

In [12]:
target_col = 'count_list'
#predict 6 months ahead
model = models.rollingCombinedModel(df=df,predict_horizon_total=6, predict_horizon=3,
                                    feature_column_names = feature_column_names,
                                   target_column = target_col, modelName='linear',
                                   lead_target=True)

In [13]:
pred6_listing, pred6_scale_listing, pred6_samples_listing = model.make_prediction(df[feature_column_names], df[target_col])

prediction_horizon 1


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 200984.82581671004, tolerance: 31066.11181568628
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 52742.54547033273, tolerance: 51038.32710243903
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 58120.803983661346, tolerance: 37482.15055609758
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increa

prediction_horizon 2


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 208624.04951880366, tolerance: 83288.3698
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 380533.5277354107, tolerance: 88926.964042
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 76624.11679366976, tolerance: 56111.23384
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of it

prediction_horizon 3


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1532299.6108684933, tolerance: 216863.15036326533
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 406614.5373875097, tolerance: 142378.8646
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 170600.7541307034, tolerance: 77212.663
  positive)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number 

In [14]:
pred6_samples_listing.shape

(50, 6)

### Step 3: Save the prediction samples for further use

In [15]:
for i in range(6):
    save = pd.DataFrame(columns=['number of sales', 'number of listings'])
    save['number of sales'] = pred6_samples_sale[:,i]
    save['number of listings'] = pred6_samples_listing[:,i]
    save.to_csv('outputs/denver/'+str(i+1)+'_month_market_level_forecast_samples.csv', index=False)